# Distillation. Student Inference


## Imports


In [40]:
import os
import gc
import numpy as np
import pandas as pd
from PIL import Image

import cv2
import timm
import torch
import torch.nn as nn
import torch.nn.functional as F
import pytorch_lightning as pl
from tqdm import tqdm
from torch.optim import AdamW
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import CosineAnnealingLR
from transformers import Dinov2Model, Dinov2Config

import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)
print(f"PyTorch: {torch.__version__}")
print(
    f"Device: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU'}")

PyTorch: 2.9.1+cu128
Device: NVIDIA GeForce RTX 5050 Laptop GPU


In [41]:
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        # print(os.path.join(dirname, filename))
        pass

In [42]:
cpu_count = os.cpu_count()
NUM_WORKERS = 0

LR = 1e-4
EPOCHS = 25
N_FOLDS = 5
GRAD_ACCUM = 1
BATCH_SIZE = 16
DROPOUT_RATE = 0.3
# Weight for distillation loss
# Loss = DISTILL_ALPHA * Distillation_Loss + (1 - DISTILL_ALPHA) * Hard_Loss
DISTILL_ALPHA = 0.5
WEIGHT_DECAY = 0.05
HIDDEN_RATIO = 0.5
TRAIN_SPLIT_RATIO = 0.02  # Used if N_FOLDS = 0

MODEL = 'facebook/dinov2-base'
CHECKPOINTS_DIR = f"./kaggle/input/2head/"
WEIGHTS_PATH = f"{CHECKPOINTS_DIR}{MODEL.replace('/', '_')}.pth"
PROJECT_NAME = "csiro-image2biomass-prediction"
# Whether to use OOF soft targets or 100% ensemble soft targets
USE_OOF_SOFT_TARGETS = False

# Each patch is 1000x1000, resize to 768x768 for vision transformers
SIZE = 768
USE_LOG_TARGET = True     # Whether to use log1p transformation on target variable
FUSION_METHOD = 'gating'  # ('concat', 'mean', 'max') OR 'gating'

DESCRIPTION = "kaggle" + \
    (f"_train{TRAIN_SPLIT_RATIO}" if N_FOLDS == 0 else f"_train[{N_FOLDS}]Folds") + (
        f"_log" if USE_LOG_TARGET else "") + f"_fusion-{FUSION_METHOD}"
DESCRIPTION_FULL = MODEL + "-" + DESCRIPTION + \
    f"_epochs{EPOCHS}_bs{BATCH_SIZE}_gradacc{GRAD_ACCUM}_lr{LR}_wd{WEIGHT_DECAY}_dr{DROPOUT_RATE}_hr{HIDDEN_RATIO}"
SUBMISSION_NAME = f"{DESCRIPTION_FULL}_submission.csv"
SUBMISSION_ENSEMBLE_NAME = f"{DESCRIPTION_FULL}_ensemble_submission.csv"
SUBMISSION_MSG = DESCRIPTION_FULL.replace("_", " ")

SEED = 1488
torch.manual_seed(SEED)
np.random.seed(SEED)
pl.seed_everything(SEED)

print("DESCRIPTION_FULL:", DESCRIPTION_FULL)
print(f"Effective batch size: {BATCH_SIZE * GRAD_ACCUM}")

Seed set to 1488


DESCRIPTION_FULL: facebook/dinov2-base-kaggle_train[5]Folds_log_fusion-gating_epochs25_bs16_gradacc1_lr0.0001_wd0.05_dr0.3_hr0.5
Effective batch size: 16


In [43]:
# setting device on GPU if available, else CPU
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', DEVICE)
print('NUM_WORKERS:', NUM_WORKERS)
print()

# Additional Info when using cuda
if DEVICE.type == 'cuda':
    # clean GPU memory
    torch.cuda.empty_cache()
    gc.collect()

    # torch.set_float32_matmul_precision('high')

    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3, 1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3, 1), 'GB')

Using device: cuda
NUM_WORKERS: 0

NVIDIA GeForce RTX 5050 Laptop GPU
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB


## Model Architecture


In [44]:
labels = [
    "Dry_Clover_g",
    "Dry_Dead_g",
    "Dry_Green_g",
    "Dry_Total_g",
    "GDM_g"
]

weights = {
    'Dry_Green_g': 0.1,
    'Dry_Dead_g': 0.1,
    'Dry_Clover_g': 0.1,
    'GDM_g': 0.2,
    'Dry_Total_g': 0.5,
}


def competition_metric(y_true, y_pred) -> float:
    """Function to calculate the competition's official evaluation metric (weighted R2 score)."""
    weights_array = np.array([weights[l] for l in labels])

    # Align with this calculation method
    y_weighted_mean = np.average(y_true, weights=weights_array, axis=1).mean()

    # For ss_res and ss_tot, also take the weighted average on axis=1, then the mean of the result
    ss_res = np.average((y_true - y_pred)**2,
                        weights=weights_array, axis=1).mean()
    ss_tot = np.average((y_true - y_weighted_mean)**2,
                        weights=weights_array, axis=1).mean()

    return 1 - ss_res / ss_tot

In [45]:
class BiomassTeacherModelPatches(pl.LightningModule):
    """
    Dual-head teacher model with patch-level predictions using DINOv2.
    - Image-only head: for distillation to student
    - Privileged head: with tabular features for maximum accuracy
    """

    def __init__(
        self,
        backbone_name: str = 'facebook/dinov2-base',
        tabular_dim: int = 10,
        num_targets: int = 3,
        lr: float = 1e-4,
        weight_decay: float = 1e-5,
        hidden_ratio: float = 0.5,
        dropout: float = 0.2,
        fusion_method: str = 'gating',
        use_log_target: bool = True,
        tabular_dropout_prob: float = 0.3,
        lambda_cons: float = 0.5,
        pretrained_backbone: bool = True,
        backbone_weights_path: str | None = None,
    ):
        """
        Args:
            backbone_name: DINOv2 model name
            tabular_dim: dimension of tabular features
            num_targets: number of regression targets (3)
            lr: learning rate
            weight_decay: weight decay for optimizer
            hidden_ratio: ratio for hidden layer size
            dropout: dropout probability
            fusion_method: how to use tabular features ('gating' or 'concat')
            use_log_target: if True, predict log1p transformed targets
            tabular_dropout_prob: probability of zeroing tabular features during training
            lambda_cons: weight for consistency loss between image-only and privileged heads
            pretrained_backbone: whether to download/use pretrained backbone weights
            backbone_weights_path: optional path to local backbone weights (state_dict)
        """
        super().__init__()
        self.save_hyperparameters()

        # Load DINOv2 backbone (supports offline)
        if pretrained_backbone:
            self.backbone = Dinov2Model.from_pretrained(backbone_name)
        else:
            try:
                config = Dinov2Config.from_pretrained(
                    backbone_name, local_files_only=True)
            except Exception:
                config = Dinov2Config()
            self.backbone = Dinov2Model(config)
            if backbone_weights_path and os.path.exists(backbone_weights_path):
                state_dict = torch.load(
                    backbone_weights_path, map_location='cpu')
                missing, unexpected = self.backbone.load_state_dict(
                    state_dict, strict=False)
                if missing or unexpected:
                    print(
                        f"Backbone load - missing: {missing}, unexpected: {unexpected}")

        self.backbone.train()

        self.hidden_dim = self.backbone.config.hidden_size
        self.patch_size = self.backbone.config.patch_size

        self.lr = lr
        self.weight_decay = weight_decay
        self.fusion_method = fusion_method
        self.use_log_target = use_log_target
        self.tabular_dropout_prob = tabular_dropout_prob
        self.lambda_cons = lambda_cons

        # Mode for prediction: 'img' for image-only, 'priv' for privileged
        self.prediction_mode = 'priv'

        # Patch-level MLPs (shared across all patches)
        hidden_size = max(32, int(self.hidden_dim * hidden_ratio))

        def make_patch_head():
            """MLP for patch-level prediction"""
            return nn.Sequential(
                nn.Linear(self.hidden_dim, hidden_size),
                nn.LayerNorm(hidden_size),
                nn.ReLU(inplace=True),
                nn.Dropout(dropout),
                nn.Linear(hidden_size, 1)
            )

        # IMAGE-ONLY heads (for distillation to student)
        self.img_head_green = make_patch_head()
        self.img_head_clover = make_patch_head()
        self.img_head_dead = make_patch_head()

        # PRIVILEGED heads (with tabular features for best accuracy)
        self.priv_head_green = make_patch_head()
        self.priv_head_clover = make_patch_head()
        self.priv_head_dead = make_patch_head()

        # Tabular features fusion for privileged branch
        if self.fusion_method == 'gating':
            self.tabular_gate = nn.Sequential(
                nn.Linear(tabular_dim, hidden_size),
                nn.ReLU(inplace=True),
                nn.Linear(hidden_size, 1),
                nn.Sigmoid()
            )
        elif self.fusion_method == 'concat':
            self.fusion_layer = nn.Sequential(
                nn.Linear(3 + tabular_dim, hidden_size),
                nn.ReLU(inplace=True),
                nn.Dropout(dropout),
                nn.Linear(hidden_size, 3)
            )

        self.validation_step_outputs = []

        msg = (
            "Dual-Head Patch-level Teacher Model initialized:\n"
            f"backbone={backbone_name}, hidden_dim={self.hidden_dim}, patch_size={self.patch_size},\n"
            f"fusion_method={fusion_method}, use_log_target={use_log_target},\n"
            f"tabular_dropout_prob={tabular_dropout_prob}, lambda_cons={lambda_cons}"
        )
        print(msg)

In [46]:
config = Dinov2Config.from_pretrained(MODEL)
temp_backbone = Dinov2Model.from_pretrained(MODEL, config=config)

In [54]:
# # save model weights to file
# torch.save(temp_backbone.state_dict(), WEIGHTS_PATH)

In [47]:
inputs_size = config.image_size
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

In [48]:
SIZE = inputs_size
print(f"Backbone expected input size: {inputs_size}, using SIZE={SIZE}")
print(f"Backbone expected mean: {mean}, std: {std}")

# Get backbone output dimension
with torch.no_grad():
    dummy = torch.randn(1, 3, SIZE, SIZE)
    outputs = temp_backbone(dummy)
    feat_dim = outputs.last_hidden_state.sum(
        dim=1).shape[1]  # Average pooling
    print(feat_dim)

Backbone expected input size: 518, using SIZE=518
Backbone expected mean: [0.485, 0.456, 0.406], std: [0.229, 0.224, 0.225]
768


In [53]:
student_val_transform = transforms.Compose([
    transforms.Resize((SIZE, SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std)
])

In [55]:
# TTA helpers
TTA_TYPES = ['id', 'hflip', 'vflip', 'hvflip']


def apply_tta(left: torch.Tensor, right: torch.Tensor, tta: str) -> tuple[torch.Tensor, torch.Tensor]:
    """Apply simple flip-based TTA to both patches."""
    if tta == 'hflip':
        return torch.flip(left, dims=[2]), torch.flip(right, dims=[2])
    if tta == 'vflip':
        return torch.flip(left, dims=[1]), torch.flip(right, dims=[1])
    if tta == 'hvflip':
        return torch.flip(left, dims=[1, 2]), torch.flip(right, dims=[1, 2])
    return left, right

In [56]:
def stack_targets_from_preds(preds_3: torch.Tensor) -> torch.Tensor:
    """Given [B,3] (clover, dead, green) produce [B,5] ordered targets."""
    clover = preds_3[:, 0]
    dead = preds_3[:, 1]
    green = preds_3[:, 2]
    total = green + dead + clover
    gdm = clover + green
    return torch.stack([clover, dead, green, total, gdm], dim=1)

In [57]:
def predict_model_batch(model: BiomassTeacherModelPatches, batch: dict, tta_types: list[str]) -> torch.Tensor:
    """Run model over TTA variants and average. Returns [B,5]."""
    model_preds = []
    for tta in tta_types:
        left_t, right_t = apply_tta(
            batch['left_image'], batch['right_image'], tta)
        tta_batch = {
            'left_image': left_t,
            'right_image': right_t,
            'tabular': batch['tabular'],
        }
        preds_3 = model.predict_step(tta_batch, 0)  # [B,3]
        model_preds.append(stack_targets_from_preds(preds_3))
    return torch.stack(model_preds, dim=0).mean(dim=0)

## Inference on Test Set


In [58]:
PATH_DATA = './kaggle/input/csiro-biomass'
STUDENT_MODELS_DIR = './kaggle/input/2head'
PATH_TEST_CSV = os.path.join(PATH_DATA, 'test.csv')
PATH_TEST_IMG = os.path.join(PATH_DATA, 'test')

In [59]:
# Load test CSV
test_df = pd.read_csv(PATH_TEST_CSV)
test_df = test_df[~test_df['target_name'].isin(['Dry_Total_g', 'GDM_g'])]

# Pivot to one row per image
test_pivot = test_df.pivot_table(
    index='image_path',
    aggfunc='first'
).reset_index()

print(f"Test set size: {len(test_pivot)}")
print(test_pivot.head())

Test set size: 1
              image_path                   sample_id   target_name
0  test/ID1001187975.jpg  ID1001187975__Dry_Clover_g  Dry_Clover_g


In [60]:
# Checkpoint discovery and loading
def parse_metric_from_filename(filename: str) -> float:
    """Extract val_comp_metric_img from filename like ...val_comp_metric_img=0.7129.ckpt."""
    try:
        metric_part = filename.split('val_comp_metric_img=')[-1]
        return float(metric_part.replace('.ckpt', ''))
    except Exception:
        return -float('inf')

In [62]:
def load_student_model(ckpt_path: str, backbone_weights_path: str | None = None) -> BiomassTeacherModelPatches:
    """Load model from checkpoint without internet (uses saved weights)."""
    checkpoint = torch.load(ckpt_path, map_location='cpu')
    hparams = checkpoint['hyper_parameters']

    model = BiomassTeacherModelPatches(
        backbone_name=hparams['backbone_name'],
        tabular_dim=hparams.get('tabular_dim', 0),
        num_targets=hparams['num_targets'],
        lr=hparams['lr'],
        weight_decay=hparams['weight_decay'],
        hidden_ratio=hparams['hidden_ratio'],
        dropout=hparams['dropout'],
        fusion_method=hparams['fusion_method'],
        use_log_target=hparams['use_log_target'],
        tabular_dropout_prob=hparams.get('tabular_dropout_prob', 0.0),
        lambda_cons=hparams.get('lambda_cons', 0.0),
        pretrained_backbone=False,
        backbone_weights_path=backbone_weights_path,
    )

    model.load_state_dict(checkpoint['state_dict'])
    model.prediction_mode = 'img'  # image-only branch for inference
    model.eval()
    model.to(DEVICE)
    return model

In [63]:
# Discover fold checkpoints and best overall
ckpt_files = sorted([
    f for f in os.listdir(STUDENT_MODELS_DIR) if f.endswith('.ckpt')
])
print(f"Found {len(ckpt_files)} student checkpoints:")
for f in ckpt_files:
    print(f"  - {f}")

Found 5 student checkpoints:
  - f0dinov2-base-local_train(5)Folds_log_fusion-gating_epochs30_bs4_gradacc4_lr3e-05_wd0.05_dr0.2_hr0.5-fold0-epoch=23-val_loss_img=0.0000-val_comp_metric_img=0.7129.ckpt
  - f1dinov2-base-local_train(5)Folds_log_fusion-gating_epochs30_bs4_gradacc4_lr3e-05_wd0.05_dr0.2_hr0.5-fold1-epoch=24-val_loss_img=0.0000-val_comp_metric_img=0.726.ckpt
  - f2dinov2-base-local_train(5)Folds_log_fusion-gating_epochs30_bs4_gradacc4_lr3e-05_wd0.05_dr0.2_hr0.5-fold2-epoch=28-val_loss_img=0.0000-val_comp_metric_img=0.773.ckpt
  - f3dinov2-base-local_train(5)Folds_log_fusion-gating_epochs30_bs4_gradacc4_lr3e-05_wd0.05_dr0.2_hr0.5-fold3-epoch=25-val_loss_img=0.0000-val_comp_metric_img=0.725.ckpt
  - f4dinov2-base-local_train(5)Folds_log_fusion-gating_epochs30_bs4_gradacc4_lr3e-05_wd0.05_dr0.2_hr0.5-fold4-epoch=26-val_loss_img=0.0000-val_comp_metric_img=0.7169.ckpt


In [64]:
# Select 5 folds: filenames starting with f{fold}
fold_ckpts = []
for fold_id in range(N_FOLDS):
    candidates = [f for f in ckpt_files if f.startswith(f"f{fold_id}")]
    if not candidates:
        continue
    # pick best metric per fold
    candidates.sort(key=parse_metric_from_filename, reverse=True)
    fold_ckpts.append(os.path.join(STUDENT_MODELS_DIR, candidates[0]))

In [65]:
# Best overall by metric
best_ckpt = None
if ckpt_files:
    best_ckpt = os.path.join(
        STUDENT_MODELS_DIR,
        sorted(ckpt_files, key=parse_metric_from_filename, reverse=True)[0]
    )

print("Selected fold checkpoints:")
for p in fold_ckpts:
    print(f"  {os.path.basename(p)}")
print(
    f"Best checkpoint: {os.path.basename(best_ckpt) if best_ckpt else 'None'}")

Selected fold checkpoints:
  f0dinov2-base-local_train(5)Folds_log_fusion-gating_epochs30_bs4_gradacc4_lr3e-05_wd0.05_dr0.2_hr0.5-fold0-epoch=23-val_loss_img=0.0000-val_comp_metric_img=0.7129.ckpt
  f1dinov2-base-local_train(5)Folds_log_fusion-gating_epochs30_bs4_gradacc4_lr3e-05_wd0.05_dr0.2_hr0.5-fold1-epoch=24-val_loss_img=0.0000-val_comp_metric_img=0.726.ckpt
  f2dinov2-base-local_train(5)Folds_log_fusion-gating_epochs30_bs4_gradacc4_lr3e-05_wd0.05_dr0.2_hr0.5-fold2-epoch=28-val_loss_img=0.0000-val_comp_metric_img=0.773.ckpt
  f3dinov2-base-local_train(5)Folds_log_fusion-gating_epochs30_bs4_gradacc4_lr3e-05_wd0.05_dr0.2_hr0.5-fold3-epoch=25-val_loss_img=0.0000-val_comp_metric_img=0.725.ckpt
  f4dinov2-base-local_train(5)Folds_log_fusion-gating_epochs30_bs4_gradacc4_lr3e-05_wd0.05_dr0.2_hr0.5-fold4-epoch=26-val_loss_img=0.0000-val_comp_metric_img=0.7169.ckpt
Best checkpoint: f2dinov2-base-local_train(5)Folds_log_fusion-gating_epochs30_bs4_gradacc4_lr3e-05_wd0.05_dr0.2_hr0.5-fold2-ep

In [67]:
# Load models WITHOUT internet (offline inference on Kaggle)
student_models = []
for ckpt_path in fold_ckpts:
    print(f"\nLoading fold model: {os.path.basename(ckpt_path)}")
    student_models.append(load_student_model(
        ckpt_path, backbone_weights_path=WEIGHTS_PATH))

if best_ckpt:
    print(f"\nLoading best model: {os.path.basename(best_ckpt)}")
    student_models.append(load_student_model(
        best_ckpt, backbone_weights_path=WEIGHTS_PATH))

print(f"\nSuccessfully loaded {len(student_models)} student models")
print("Ready for offline inference on Kaggle!")


Loading fold model: f0dinov2-base-local_train(5)Folds_log_fusion-gating_epochs30_bs4_gradacc4_lr3e-05_wd0.05_dr0.2_hr0.5-fold0-epoch=23-val_loss_img=0.0000-val_comp_metric_img=0.7129.ckpt
Dual-Head Patch-level Teacher Model initialized:
backbone=facebook/dinov2-base, hidden_dim=768, patch_size=14,
fusion_method=gating, use_log_target=True,
tabular_dropout_prob=0.3, lambda_cons=0.5

Loading fold model: f1dinov2-base-local_train(5)Folds_log_fusion-gating_epochs30_bs4_gradacc4_lr3e-05_wd0.05_dr0.2_hr0.5-fold1-epoch=24-val_loss_img=0.0000-val_comp_metric_img=0.726.ckpt
Dual-Head Patch-level Teacher Model initialized:
backbone=facebook/dinov2-base, hidden_dim=768, patch_size=14,
fusion_method=gating, use_log_target=True,
tabular_dropout_prob=0.3, lambda_cons=0.5

Loading fold model: f2dinov2-base-local_train(5)Folds_log_fusion-gating_epochs30_bs4_gradacc4_lr3e-05_wd0.05_dr0.2_hr0.5-fold2-epoch=28-val_loss_img=0.0000-val_comp_metric_img=0.773.ckpt
Dual-Head Patch-level Teacher Model initial

In [68]:
# Create test dataset
class BiomassTestDataset(Dataset):
    """Test dataset for inference - no targets needed."""

    def __init__(self, df: pd.DataFrame, img_dir: str, transform=None):
        self.df = df.reset_index(drop=True)
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]

        # Load image
        img_path = os.path.join(
            self.img_dir, row['image_path'].replace('test/', ''))
        image = cv2.imread(img_path)

        if image is None:
            raise FileNotFoundError(f"Cannot load image: {img_path}")

        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Split into left and right patches
        h, w, c = image.shape
        mid_w = w // 2

        left_patch = image[:, :mid_w, :]
        right_patch = image[:, mid_w:, :]

        # Convert to PIL
        left_pil = Image.fromarray(left_patch)
        right_pil = Image.fromarray(right_patch)

        # Apply transforms
        if self.transform:
            left_tensor = self.transform(left_pil)
            right_tensor = self.transform(right_pil)
        else:
            left_tensor = transforms.ToTensor()(left_pil)
            right_tensor = transforms.ToTensor()(right_pil)

        return {
            'left_image': left_tensor,
            'right_image': right_tensor,
            'image_id': row['image_path'].split('/')[-1].replace('.jpg', ''),
            'state': row.get('State', None)
        }

In [69]:
# Create test dataloader
test_dataset = BiomassTestDataset(
    df=test_pivot,
    img_dir=PATH_TEST_IMG,
    transform=student_val_transform
)

test_loader = DataLoader(
    test_dataset,
    batch_size=BATCH_SIZE * 2,
    shuffle=False,
    num_workers=min(NUM_WORKERS, 4),
    pin_memory=True if torch.cuda.is_available() else False
)

print(f"Test loader created: {len(test_loader)} batches")

Test loader created: 1 batches


In [70]:
# Run inference on test set with TTA and WA rule
print("Running inference on test set...")

all_predictions = []
all_image_ids = []
all_states = []

if len(student_models) == 0:
    raise RuntimeError("No student models loaded for inference")

tabular_dim = student_models[0].hparams.tabular_dim

print(f"Using tabular_dim={tabular_dim} for inference")

Running inference on test set...
Using tabular_dim=21 for inference


In [72]:
with torch.no_grad():
    for batch_idx, batch in enumerate(tqdm(test_loader, desc="Inference")):
        states = batch.pop('state')
        all_states.extend(states)

        # Move to device
        batch['left_image'] = batch['left_image'].to(DEVICE)
        batch['right_image'] = batch['right_image'].to(DEVICE)
        batch['tabular'] = torch.zeros(
            batch['left_image'].size(0), tabular_dim, device=DEVICE)

        # Ensemble predictions from all models with TTA
        batch_preds_list = []
        for model in student_models:
            model_preds = predict_model_batch(model, batch, TTA_TYPES)  # [B,5]
            batch_preds_list.append(model_preds.cpu())

        # Average predictions across models
        batch_preds_avg = torch.stack(
            batch_preds_list, dim=0).mean(dim=0)  # [B,5]

        # WA rule: if state == "WA" then dead_g = 0
        batch_preds_np = batch_preds_avg.numpy()
        for i, st in enumerate(states):
            if st == 'WA':
                # indices: [clover, dead, green, total, gdm]
                batch_preds_np[i, 1] = 0.0
                batch_preds_np[i, 3] = batch_preds_np[i, 0] + \
                    batch_preds_np[i, 2]

        all_predictions.append(batch_preds_np)
        all_image_ids.extend(batch['image_id'])

# Concatenate all predictions
all_predictions_array = np.concatenate(all_predictions, axis=0)
print(f"Predictions shape: {all_predictions_array.shape}")
print(f"Image IDs count: {len(all_image_ids)}")

Inference:   0%|          | 0/1 [00:00<?, ?it/s]


TypeError: default_collate: batch must contain tensors, numpy arrays, numbers, dicts or lists; found <class 'NoneType'>

In [ ]:
# Format submission CSV
# Columns order: Dry_Clover_g, Dry_Dead_g, Dry_Green_g, Dry_Total_g, GDM_g
target_names = ['Dry_Clover_g', 'Dry_Dead_g',
                'Dry_Green_g', 'Dry_Total_g', 'GDM_g']

submission_rows = []

for img_idx, image_id in enumerate(all_image_ids):
    predictions = all_predictions_array[img_idx]  # [5] values for 5 targets

    for target_idx, target_name in enumerate(target_names):
        sample_id = f"{image_id}__{target_name}"
        target_value = float(predictions[target_idx])

        submission_rows.append({
            'sample_id': sample_id,
            'target': target_value
        })

# Create submission dataframe
submission_df = pd.DataFrame(submission_rows)

print(f"Submission shape: {submission_df.shape}")
print(f"Expected shape: ({len(test_pivot) * 5}, 2)")
print(submission_df.head(10))

In [ ]:
# Save submission
submission_df.to_csv(SUBMISSION_NAME, index=False)

print(f"Submission saved to: {SUBMISSION_NAME}")